In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#RNN

In [6]:
!pip install joblib
!pip install colorama
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from keras.layers import SimpleRNN,Dense
from keras.utils import to_categorical
import joblib
from colorama import Fore, Back, Style

model_path = '/content/drive/MyDrive/EssentialAIBootcamp/rnn-phishing/rnn.joblib'
data_path = '/content/drive/MyDrive/EssentialAIBootcamp/rnn-phishing/Phishing_Email.csv'

In [8]:
mailData = pd.read_csv(data_path)
mailData = mailData.where((pd.notnull(mailData)), '')

mailData.loc[mailData['Email Type']=='Safe Email','Email Type'] = 0
mailData.loc[mailData['Email Type']=='Phishing Email','Email Type'] = 1

In [9]:
X = mailData['Email Text']
y = mailData['Email Type']

##Tokenizer

In [11]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [12]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X)
indeksli_veri = tokenizer.texts_to_sequences(X)


word_index = tokenizer.word_index

kelime_uzunlugu = 100
sabit_uzunluk = pad_sequences(indeksli_veri, maxlen=kelime_uzunlugu)

In [13]:
y = to_categorical(y)

In [14]:
X_train, X_test, y_train, y_test = train_test_split(sabit_uzunluk,y,test_size = 0.2, random_state=1)

##Embedding

In [16]:
from keras.layers import Embedding
import numpy as np

In [17]:
model = Sequential()
model.add(Embedding(len(word_index)+1,output_dim = 128, input_length=kelime_uzunlugu))
model.add(SimpleRNN(64, activation='relu'))
model.add(Dense(2,activation='softmax'))
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])


In [18]:
model.fit(X_train,y_train, epochs=15,batch_size=32, validation_data=(X_test,y_test))

Epoch 1/15
467/467 [==============================] - 285s 606ms/step - loss: 0.2300 - accuracy: 0.8983 - val_loss: 0.1066 - val_accuracy: 0.9560
Epoch 2/15
467/467 [==============================] - 274s 587ms/step - loss: 0.0663 - accuracy: 0.9749 - val_loss: 0.1410 - val_accuracy: 0.9461
Epoch 3/15
467/467 [==============================] - 291s 623ms/step - loss: 0.0338 - accuracy: 0.9859 - val_loss: 0.1174 - val_accuracy: 0.9547
Epoch 4/15
467/467 [==============================] - 273s 584ms/step - loss: 0.0306 - accuracy: 0.9864 - val_loss: 0.1791 - val_accuracy: 0.9319
Epoch 5/15
467/467 [==============================] - 290s 622ms/step - loss: 0.0321 - accuracy: 0.9869 - val_loss: 0.1821 - val_accuracy: 0.9394
Epoch 6/15
467/467 [==============================] - 270s 578ms/step - loss: 0.0218 - accuracy: 0.9887 - val_loss: 0.1300 - val_accuracy: 0.9590
Epoch 7/15
467/467 [==============================] - 272s 583ms/step - loss: 0.0221 - accuracy: 0.9885 - val_loss: 0.1591 -

In [19]:
loss, accuracy = model.evaluate(X_test, y_test)
print("Kayıp : ",loss)
print("Başarı oranı : ",accuracy)

117/117 [==============================] - 2s 14ms/step - loss: 0.1886 - accuracy: 0.9606
Kayıp :  0.1885591745376587
Başarı oranı :  0.9605898261070251


In [20]:
model.save(model_path)

##Real Time Test

In [21]:
import tensorflow as tf

model = tf.keras.models.load_model(model_path)

In [22]:
while True:
  giris = input("Email mesajını giriniz : ")
  sayısal_kelime = tokenizer.texts_to_sequences([giris])
  sabit_uzunluk = pad_sequences(sayısal_kelime, maxlen=kelime_uzunlugu)
  tahmin = model.predict(sabit_uzunluk)
  if np.argmax(tahmin) == 1:
    print(Fore.RED, 'Bu bir kimlik avı emaili olabilir.')
  else:
    print(Fore.GREEN,'Bu bir güvenli emaildir.')
  print(Fore.WHITE, 'Doğruluk puanı : ',tahmin[0][np.argmax(tahmin)])

Email mesajını giriniz : Easy qualifying, prompt, courteous service, low rates! Don't wait for interest rates to go up again, lock in YOUR low rate now!
1/1 [==============================] - 0s 195ms/step
 Bu bir kimlik avı emaili olabilir.
 Doğruluk puanı :  0.9999993


KeyboardInterrupt: Interrupted by user

#Arayüz


In [25]:
class PhishingDetectorApp(QWidget):
    # ...

    def analyze_mail(self):
        mail_text = self.mail_entry.toPlainText()
        result = check_phishing_with_model(mail_text)

        if result:
            self.result_label.setText('Bu mail phishing içerebilir!')
        else:
            self.result_label.setText('Bu mail güvenli görünüyor.')

def check_phishing_with_model(mail_text):
    # Modeli yükle
    #model_path = "rnn.joblib"  # Modelin dosya yolunu buraya ekleyin
    model = joblib.load(model_path)


###Streamlit çalışmadı

In [ ]:
pip install streamlit


In [3]:
!pip install --upgrade joblib


In [10]:
import streamlit as st
import joblib

def load_model():
    model_path = 'model.joblib'
    model = joblib.load(model_path)
    return model

def check_phishing_with_model(mail_text, model):
    # Modeli kullanarak maili analiz et
    # Burada mail_text'i modelin girişine uygun formatta düzenlemen gerekebilir
    # ve modelin çıkışını kontrol ederek phishing olup olmadığını belirleyebilirsin.
    # Bu adımları modelin eğitildiği kütüphaneye ve veri setine bağlı olarak ayarlamalısın.

    # Örnek:
    # result = model.predict(mail_text)

    # Daha sonra sonucu döndür
    # return result
      st.title("Phishing Mail Analiz Uygulaması")
mail_text = st.text_area("Mail Metni:")
if st.button("Analiz Et"):
    model = load_model()
    result = check_phishing_with_model(mail_text, model)
    if result:
        st.error("Bu mail phishing içerebilir!")
    else:
        st.success("Bu mail güvenli görünüyor.")


### ???

In [ ]:
import streamlit as st
import joblib

def check_phishing_with_model(mail_text):
    # Modeli yükle
   # model_path = '/content/drive/MyDrive/EssentialAIBootcamp/rnn-phishing/rnn.joblib'
    model = joblib.load(model_path)

    # Maili analiz et
    # Burada mail_text'i modelin girişine uygun formatta düzenlemen gerekebilir
    # ve modelin çıkışını kontrol ederek phishing olup olmadığını belirleyebilirsin.
    # Bu adımları modelin eğitildiği kütüphaneye ve veri setine bağlı olarak ayarlamalısın.

    # Örnek:
    # result = model.predict(mail_text)

    # Daha sonra sonucu döndür
    # return result

def main():
    st.title("Phishing-Mail")
    mail_text = st.text_area("Mail Metni:")
    if st.button("Analiz Et"):
        result = check_phishing_with_model(mail_text)
        if result:
            st.error("Bu mail phishing içerebilir!")
        else:
            st.success("Bu mail güvenli görünüyor.")

if __name__ == "__main__":
    main()


### !pip install pyngrok

In [ ]:
!pip install pyngrok

In [ ]:
from pyngrok import ngrok

# Streamlit uygulamanızı başlatın
!streamlit run my_phishing_app.py &

# Bağlantıyı alın
public_url = ngrok.connect(port='8501')

public_url
